In [1]:
import os
os.chdir("/content")
!rm -rf "AVSS"
!ls

sample_data


In [2]:
!git clone https://github.com/ArthurGaleev/AVSS.git

Cloning into 'AVSS'...
remote: Enumerating objects: 1697, done.
remote: Counting objects: 100% (432/432), done.
remote: Compressing objects: 100% (291/291), done.
remote: Total 1697 (delta 245), reused 251 (delta 131), pack-reused 1265 (from 1)
Receiving objects: 100% (1697/1697), 346.45 KiB | 5.97 MiB/s, done.
Resolving deltas: 100% (1129/1129), done.


In [3]:
import os
os.chdir("AVSS")

## Install requirements

In [4]:
!pip install uv
!uv sync

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.1/22.1 MB 119.9 MB/s eta 0:00:00
Using CPython 3.13.7
Creating virtual environment at: .venv
Resolved 145 packages in 5.43s
Prepared 142 packages in 1m 32s
Installed 142 packages in 866ms
 + annotated-types==0.7.0
 + antlr4-python3-runtime==4.9.3
 + attrs==25.4.0
 + audioop-lts==0.2.2
 + audioread==3.1.0
 + beautifulsoup4==4.14.3
 + black==25.9.0
 + bleach==6.3.0
 + certifi==2025.11.12
 + cffi==2.0.0
 + cfgv==3.5.0
 + charset-normalizer==3.4.4
 + click==8.3.1
 + coloredlogs==15.0.1
 + comet-ml==3.54.0
 + configobj==5.0.9
 + contourpy==1.3.3
 + cycler==0.12.1
 + decorator==5.2.1
 + distlib==0.4.0
 + dotenv==0.9.9
 + dulwich==0.24.10
 + everett==3.1.0
 + filelock==3.20.0
 + flatbuffers==25.9.23
 + fonttools==4.61.1
 + fsspec==2025.12.0
 + gammatone==1.0.3
 + gdown==5.2.0
 + gitdb==4.0.12
 + gitpython==3.1.45
 + humanfriendly==10.0
 + hydra-core==1.3.2
 + identify==2.6.15
 + idna==3.11
 + isort==7.0.0
 + jinja2==3.1.6
 + joblib==1.5.2
 + json

## Download models checkpoints and datasets

In [5]:
# download our dataset and all models.
# !uv run scripts/download_gdrive.py

In [6]:
import sys
sys.path.append("/content/AVSS")

### Google Drive

In [7]:
from scripts.download_gdrive import download_dataset, download_models
RTFS_MODEL_PATH="data/models/rtfs-4-reuse.pth" #Our SOTA Model
DPRNN_MODEL_PATH="data/models/dprnn.pth" #Our best dprnn checkpoint
DATASET_PATH="data/datasets"#Where you download dataset from google drive
DATASET_LINK="https://drive.google.com/uc?id=1wu_S9wpiZmpyAZosvWTCYC9W6zD6nAVe"
#If returns
GDRIVE_URLS = {
    "models": {
        # rtfs-4-reuse, links can be found in scripts/download_gdrive
        "https://drive.google.com/uc?id=1lzO_cyN0WKof1rtnCPZpN-BHcELO-85c": RTFS_MODEL_PATH,
        "https://drive.google.com/uc?id=1IU4mFseGPtac6g58jTS48GzlMI1wMJQ8":DPRNN_MODEL_PATH
    },
    "dataset": {
        DATASET_LINK:DATASET_PATH#Put your dataset link here
    }
}
download_models(GDRIVE_URLS) #if models dont download you can download them manually to data/models
download_dataset(GDRIVE_URLS)

Downloading...
From (original): https://drive.google.com/uc?id=1lzO_cyN0WKof1rtnCPZpN-BHcELO-85c
From (redirected): https://drive.google.com/uc?id=1lzO_cyN0WKof1rtnCPZpN-BHcELO-85c&confirm=t&uuid=9110336d-f9ae-4a00-9cf4-b22a29fecc89
To: /content/AVSS/data/models/rtfs-4-reuse.pth
100%|██████████| 71.2M/71.2M [00:01<00:00, 48.4MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1IU4mFseGPtac6g58jTS48GzlMI1wMJQ8
From (redirected): https://drive.google.com/uc?id=1IU4mFseGPtac6g58jTS48GzlMI1wMJQ8&confirm=t&uuid=f996f50a-783d-4556-ba56-e690cd217a32
To: /content/AVSS/data/models/dprnn.pth
100%|██████████| 31.7M/31.7M [00:00<00:00, 187MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1wu_S9wpiZmpyAZosvWTCYC9W6zD6nAVe
From (redirected): https://drive.google.com/uc?id=1wu_S9wpiZmpyAZosvWTCYC9W6zD6nAVe&confirm=t&uuid=ac3edd28-d923-437b-b201-a061d529149b
To: /content/AVSS/data/datasets.zip
100%|██████████| 6.24G/6.24G [00:53<00:00, 117MB/s]


### Yandex Disk

In [8]:
# you can also specify a link to your yandex disk zip dataset for it to download during training ot infernce, see src/datasets/yandex_download for details
# used this to check inference val/test folders
os.environ["YANDEX_DISK_URL"]="https://disk.360.yandex.ru/d/5pz96ysIZi33IQ"  # dla_dataset with small train, but whole val/test
download_name = "dla_dataset_small_av"

# used this to check inference on one folder with and w/o gt in corresponding sections
# os.environ["YANDEX_DISK_URL"]="https://disk.360.yandex.ru/d/ML0ey9j1KJOdtg"  # dla_dataset with only val in one folder
# download_name = "custom_dataset"

In [9]:
import io
import zipfile
from pathlib import Path
from urllib.parse import urlencode
import requests


data_dir = Path("/content/AVSS/data/datasets")
if not (data_dir / download_name / "audio").exists():
    try:
        data_dir.mkdir(exist_ok=True, parents=True)
        download_info = {
            "base_url": "https://cloud-api.yandex.net/v1/disk/public/resources/download?",
            "public_key": os.getenv("YANDEX_DISK_URL"),
        }
        final_url = download_info["base_url"] + urlencode(
            dict(public_key=download_info["public_key"])
        )
        response = requests.get(final_url)
        download_url = response.json()["href"]
        print("Downloading zip data...")
        download_response = requests.get(download_url)
        print("Successfully downloaded")
        zip = zipfile.ZipFile(io.BytesIO(download_response.content))
        zip.extractall(data_dir)
    except requests.exceptions.ConnectTimeout as e:
        print(f"Connection timed out: {e}")
    except requests.exceptions.ReadTimeout as e:
        print(f"Read timed out: {e}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

## DPRNN

### Inference on val/test folders

In [10]:
!uv run inference.py inferencer.save_path="pred_dprnn_small_av" \
 batch_size=7 use_pit=True inferencer.from_pretrained={DPRNN_MODEL_PATH} download_name={download_name} model=dprnn -cn=inference

Using GPU: cuda:0 with 14.64 GB free
Creating dataset:   0% 0/500 [00:00<?, ?it/s]Downloading...
From (original): https://drive.google.com/uc?id=179NgMsHo9TeZCLLtNWFVgRehDvzteMZE
From (redirected): https://drive.google.com/uc?id=179NgMsHo9TeZCLLtNWFVgRehDvzteMZE&confirm=t&uuid=c66003d6-0dfd-401d-b2fd-aaf08b9fe045
To: /content/AVSS/src/utils/lipreading/loaded_models/resnet18_dctcn.pth

  0% 0.00/211M [00:00<?, ?B/s]
  9% 19.4M/211M [00:00<00:01, 191MB/s]
 23% 49.3M/211M [00:00<00:00, 252MB/s]
 39% 81.3M/211M [00:00<00:00, 281MB/s]
 52% 110M/211M [00:00<00:00, 281MB/s] 
 68% 143M/211M [00:00<00:00, 298MB/s]
100% 211M/211M [00:00<00:00, 304MB/s]
Lipreading model has been successfully loaded
Creating dataset: 100% 500/500 [00:22<00:00, 22.18it/s]
Creating dataset:   0% 0/3000 [00:00<?, ?it/s]Downloading...
From (original): https://drive.google.com/uc?id=179NgMsHo9TeZCLLtNWFVgRehDvzteMZE
From (redirected): https://drive.google.com/uc?id=179NgMsHo9TeZCLLtNWFVgRehDvzteMZE&confirm=t&uuid=4b085

### Inference on one folder with gt

In [11]:
# !uv run inference.py inferencer.save_path="pred_dprnn_custom" \
#  batch_size=7 datasets.test=null datasets.val.part="" use_pit=True inferencer.from_pretrained={DPRNN_MODEL_PATH} download_name={download_name} model=dprnn -cn=inference

### Inference on one folder w/o gt

In [ ]:
# !uv run inference.py batch_size=7 inferencer.save_path="pred_dprnn_custom_test" \
#  datasets.val=null datasets.test.part="" use_pit=True inferencer.from_pretrained={DPRNN_MODEL_PATH} download_name={download_name} model=dprnn -cn=inference

## RTFS

### Inference on val/test folders

In [12]:
!uv run inference.py inferencer.save_path="pred_rtfs_small_av" \
 inferencer.from_pretrained={RTFS_MODEL_PATH} download_name={download_name} model=rtfs-4-reuse -cn=inference

Using GPU: cuda:0 with 14.64 GB free
Creating dataset: 100% 500/500 [00:07<00:00, 62.67it/s]
Creating dataset: 100% 3000/3000 [00:17<00:00, 170.15it/s]
RTFSModel(
  (stft): TransformSTFT()
  (audio_encoder): RTFSAudioEncoder(
    (conv): Conv2d(2, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (ap_block): RTFSBlock(
    (compressor): CompressorBlock(
      (dimension_compression_layer): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
      (tf_compression_layers): Sequential(
        (0): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), groups=128)
        (1): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), groups=128)
      )
    )
    (reconstructor): ReconstructionBlock(
      (g_fuse_layers): ModuleList(
        (0-2): 3 x TFARUnit(
          (w1_pathway): Sequential(
            (0): Conv2d(128, 128, kernel_size=(4, 4), stride=(1, 1), padding=same, groups=128)
            (1): GlobalLayerNorm(1, 128, eps=1e-08, affine=True

### Inference on one folder with gt

In [13]:
# !uv run inference.py inferencer.save_path="pred_rtfs_custom" \
#  inferencer.from_pretrained={RTFS_MODEL_PATH} datasets.test=null datasets.val.part="" download_name={download_name} model=rtfs-4-reuse -cn=inference

### Inference on one folder w/o gt

In [ ]:
# !uv run inference.py inferencer.save_path="pred_rtfs_custom_test" \
#  inferencer.from_pretrained={RTFS_MODEL_PATH} datasets.val=null datasets.test.part="" download_name={download_name} model=rtfs-4-reuse -cn=inference

## Calc metrics

In [14]:
#calculate metrics on predictions on the dla_dataset_small_av folder (download_name) of rtfs model
# download_name =

# name of the folder, where you stored models predictions
preds_folder_name = "pred_rtfs_small_av"

# should be empty if the dataset was one folder like, e.g. w/o partitions
# should be "val" or other partition name, if there were partitions in provided dataset,
# also provided pratition should contain gt
part = "val"

# should be equal to part if the dataset had partitions,
# if the dataset was one folder like: set "val" if you ran with gt, "test" if you ran w/o gt
preds_part = part

!uv run calc_metrics.py --gt_s1=data/datasets/{download_name}/audio/{part}/s1 --gt_s2=data/datasets/{download_name}/audio/{part}/s2 \
    --pred_s1=data/saved/{preds_folder_name}/{preds_part}/s1 --pred_s2=data/saved/{preds_folder_name}/{preds_part}/s2  --mix=data/datasets/{download_name}/audio/{part}/mix

Calculating Metrics...: 100% 500/500 [02:04<00:00,  4.01it/s]
Average Metrics:
STOI: 0.8330
PESQ: 1.9828
SI-SDR-I: 8.4503
SI-SNR-I: 8.4524


## Bonus, RTFS Model (num params + secs per batch of size 1)

In [15]:
import time
start = time.time()

!uv run inference.py inferencer.save_path="" \
    batch_size=1 metrics.inference=[]\
    inferencer.from_pretrained={RTFS_MODEL_PATH} \
    download_name=dla_dataset_small_av\
    model=rtfs-4-reuse inferencer.device=cpu  datasets.test=null \
    +datasets.val.limit=1 -cn=inference

end = time.time()

Creating dataset: 100% 500/500 [00:09<00:00, 54.63it/s]
RTFSModel(
  (stft): TransformSTFT()
  (audio_encoder): RTFSAudioEncoder(
    (conv): Conv2d(2, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (ap_block): RTFSBlock(
    (compressor): CompressorBlock(
      (dimension_compression_layer): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
      (tf_compression_layers): Sequential(
        (0): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), groups=128)
        (1): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), groups=128)
      )
    )
    (reconstructor): ReconstructionBlock(
      (g_fuse_layers): ModuleList(
        (0-2): 3 x TFARUnit(
          (w1_pathway): Sequential(
            (0): Conv2d(128, 128, kernel_size=(4, 4), stride=(1, 1), padding=same, groups=128)
            (1): GlobalLayerNorm(1, 128, eps=1e-08, affine=True)
            (2): Sigmoid()
          )
          (w2_pathway): Sequential(
            (0): Co

In [16]:
print("Total elapsed:", end - start, "seconds")
print("Per batch Time: ", 9, " seconds") #can be seen on the tqdm bar: val: 100% 1/1 [00:09<00:00,  9.09s/it]
#you can increase batch_size and divide by batch_size to see time the implementation

Total elapsed: 30.08139443397522 seconds
Per batch Time:  9  seconds
